In [1]:
import msal
import requests

In [15]:
# Load credentials from a JSON file
with open('credentials.json') as f:
    credentials = json.load(f)

client_id = credentials['client_id']
client_secret = credentials['client_secret']
redirect_uri = 'http://localhost'  # Redirect URI you set in the app registration
authority = 'https://login.microsoftonline.com/common'
scopes = ['Files.ReadWrite.All']

In [16]:
# Step 1: Create a confidential client application
app = msal.ConfidentialClientApplication(client_id, authority=authority, client_credential=client_secret)

In [17]:
# Step 2: Get the authorization URL
auth_url = app.get_authorization_request_url(scopes, redirect_uri=redirect_uri)
print(f"Please go to this URL and authorize the app: {auth_url}")

Please go to this URL and authorize the app: https://login.microsoftonline.com/common/oauth2/v2.0/authorize?client_id=eee9040c-6dc0-4cc6-858b-56348f716240&response_type=code&redirect_uri=http%3A%2F%2Flocalhost&scope=Files.ReadWrite.All+offline_access+openid+profile


In [20]:
# Step 3: After the user authorizes the app, they will be redirected to the redirect URI with a code
# You need to capture that code and use it to get an access token
code = input("Enter the authorization code: ")

In [21]:
# Step 4: Get the access token
result = app.acquire_token_by_authorization_code(code, scopes=scopes, redirect_uri=redirect_uri)


In [22]:
if 'access_token' in result:
    access_token = result['access_token']
    print("Access token acquired successfully.")
else:
    print("Failed to acquire access token.")
    print(result)
    exit()

Access token acquired successfully.


In [23]:
# Define headers for the API requests
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

In [24]:
# Define the file path and workbook URL
file_path = '/Documents/_work/_employee/Logistik/Lieferando/file.xlsx'
workbook_url = f'https://graph.microsoft.com/v1.0/me/drive/root:{file_path}:/workbook'


In [25]:
# Get the workbook
response = requests.get(workbook_url, headers=headers)
workbook = response.json()
print(workbook)

{'@odata.context': "https://graph.microsoft.com/v1.0/$metadata#users('serkosif%40outlook.de')/drive/root/workbook/$entity", '@odata.id': "/users('serkosif%40outlook.de')/drive/root:/Documents/_work/_employee/Logistik/Lieferando/file.xlsx:/workbook/"}


In [26]:
# Example: Update a cell value
update_url = f'{workbook_url}/worksheets/Sheet1/range(address=\'A1\')'
update_data = {
    "values": [["Updated Value"]]
}

In [27]:
response = requests.patch(update_url, headers=headers, json=update_data)
if response.status_code == 200:
    print("Cell updated successfully.")
else:
    print("Failed to update cell.")
    print(response.json())

Cell updated successfully.
